# Advertising Budget Allocation Optimization

This notebook demonstrates an optimization workflow using the `Advertising.csv` dataset. We will:

1. Load the dataset (TV, Radio, Newspaper, Sales).
2. Fit a simple linear regression `Sales ~ TV + Radio + Newspaper`.
3. Use the regression coefficients to create a linear objective: predicted sales as a linear function of ad spend.
4. Formulate and solve an LP with **PuLP** to maximize predicted sales subject to a total advertising budget.

Deliverable: a reproducible notebook that trains the model and solves the optimization problem.

In [ ]:
# 1) Setup & load data
import pandas as pd
import numpy as np
import os

DATA_PATH = "Advertising.csv"
assert os.path.exists(DATA_PATH), f"Expected dataset at {DATA_PATH} - please ensure the file is in the same folder as the notebook."

df = pd.read_csv(DATA_PATH)
print("Loaded dataset with shape:", df.shape)
df.head()

In [ ]:
# Quick EDA: summary statistics
df.describe().T

In [ ]:
# Simple scatter plots to inspect relationships
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.scatter(df['TV'], df['Sales'], s=8)
plt.xlabel('TV Spend')
plt.ylabel('Sales')
plt.title('TV vs Sales')

plt.subplot(1,3,2)
plt.scatter(df['Radio'], df['Sales'], s=8)
plt.xlabel('Radio Spend')
plt.title('Radio vs Sales')

plt.subplot(1,3,3)
plt.scatter(df['Newspaper'], df['Sales'], s=8)
plt.xlabel('Newspaper Spend')
plt.title('Newspaper vs Sales')

plt.tight_layout()
plt.show()

In [ ]:
# 2) Fit a linear regression model: Sales ~ TV + Radio + Newspaper
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X = df[['TV','Radio','Newspaper']].copy()
y = df['Sales'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('R2 (test):', r2_score(y_test, y_pred))
print('RMSE (test):', mean_squared_error(y_test, y_pred, squared=False))

coeffs = dict(zip(['TV','Radio','Newspaper'], model.coef_))
intercept = float(model.intercept_)
print('\nLearned linear model:') 
print(f'  Sales = {intercept:.4f} + ' + ' + '.join([f'{coeffs[v]:.4f}*{v}' for v in coeffs]))

In [ ]:
# 3) Formulate LP: maximize predicted Sales = intercept + coef_TV*TV + coef_R*Radio + coef_N*Newspaper
# Decision variables: spend on TV, Radio, Newspaper (continuous, >=0)
# Constraint: total spend <= BUDGET (user-defined)
# Optional: upper bounds per channel (e.g., can't spend more than historical max * factor)

!pip install pulp --quiet

import pulp

# Coefficients from regression
coef_TV = float(coeffs['TV'])
coef_Radio = float(coeffs['Radio'])
coef_Newspaper = float(coeffs['Newspaper'])
intercept = float(intercept)

print("Regression coefficients:", coef_TV, coef_Radio, coef_Newspaper)
print("Intercept:", intercept)

# Budget parameter (you can change this)
BUDGET = 300.0  # total advertising budget (same units as dataset, e.g., thousand dollars)
print(f"Using total budget = {BUDGET}")

# Optional: set channel upper bounds (e.g., historical max observed)
max_TV = df['TV'].max() * 2
max_Radio = df['Radio'].max() * 2
max_News = df['Newspaper'].max() * 2

# Build LP
prob = pulp.LpProblem('AdBudgetAllocation', pulp.LpMaximize)
tv = pulp.LpVariable('TV_spend', lowBound=0, upBound=max_TV, cat='Continuous')
radio = pulp.LpVariable('Radio_spend', lowBound=0, upBound=max_Radio, cat='Continuous')
news = pulp.LpVariable('Newspaper_spend', lowBound=0, upBound=max_News, cat='Continuous')

# Objective: maximize predicted sales (intercept is constant; we can include it or omit as it doesn't affect argmax)
prob += coef_TV * tv + coef_Radio * radio + coef_Newspaper * news, 'Predicted_Sales_linear_component'

# Budget constraint
prob += tv + radio + news <= BUDGET, 'TotalBudget'

# (Optional) add a minimal spend requirement or discrete constraints if desired
prob.writeLP('ad_allocation.lp')
prob.solve(pulp.PULP_CBC_CMD(msg=False))

print('Status:', pulp.LpStatus[prob.status])
tv_opt = tv.varValue
radio_opt = radio.varValue
news_opt = news.varValue
predicted_sales = intercept + coef_TV*tv_opt + coef_Radio*radio_opt + coef_Newspaper*news_opt

print(f'Optimal TV spend: {tv_opt:.2f}')
print(f'Optimal Radio spend: {radio_opt:.2f}')
print(f'Optimal Newspaper spend: {news_opt:.2f}')
print(f'Predicted Sales (using linear model): {predicted_sales:.4f}')

In [ ]:
# 4) Insight: contribution per unit spend (marginal effect)
contrib = pd.Series({'TV':coef_TV, 'Radio':coef_Radio, 'Newspaper':coef_Newspaper})
contrib = contrib.sort_values(ascending=False)
print('Marginal contribution to Sales per unit spend:')
print(contrib)

# Show allocation as DataFrame
alloc = pd.DataFrame([{'channel':'TV','spend':tv_opt, 'marginal':coef_TV},
                      {'channel':'Radio','spend':radio_opt, 'marginal':coef_Radio},
                      {'channel':'Newspaper','spend':news_opt, 'marginal':coef_Newspaper}])
alloc['spend_pct'] = 100 * alloc['spend'] / alloc['spend'].sum()
alloc

In [ ]:
# Pie chart of allocation
import matplotlib.pyplot as plt

labels = alloc['channel']
sizes = alloc['spend']

plt.figure(figsize=(6,4))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title('Optimal Budget Allocation')
plt.axis('equal')
plt.show()

## Conclusion

We fitted a simple linear regression to predict `Sales` from ad spends on TV, Radio, and Newspaper. Using the learned marginal effects (coefficients), we formulated and solved a linear program to allocate a fixed advertising budget to maximize predicted sales. The solution allocates the budget to the channels with the highest marginal contribution to sales.

### Notes & next steps
- This approach uses a linear approximation (OLS). If the true response is nonlinear, consider a more complex model or include interaction/quadric terms. 
- We added upper bounds to channels based on historical maxima to keep solutions realistic. 
- You can run sensitivity analysis by varying `BUDGET` or adding constraints (minimum spends, integer constraints, channel ratios, etc.).